In [1]:
#%% Importing modules and data
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import quandl as qd
import seaborn as sns
import matplotlib.dates as dates
import matplotlib.ticker as ticker
from pandas_datareader import data
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import os
import glob

In [2]:
'''
Fields
market_cap_usd
name
price_usd
symbol
'''
exclusion_list = []# ['BCC','XRP','USDT']
capital = 20000

response = requests.get('https://api.coinmarketcap.com/v1/ticker/')
json_data = json.loads(response.text)
i = 0
crypto_df = {}
for cc in json_data:
    temp_df = pd.DataFrame(cc, index = [i])[['symbol','name','market_cap_usd','price_usd']]
    if len(crypto_df) == 0:
        crypto_df = temp_df
    else:
        crypto_df = pd.concat([crypto_df,temp_df],axis = 0)
    i += 1
    
crypto_df['market_cap_usd'] = pd.to_numeric(crypto_df['market_cap_usd'])
crypto_df['price_usd'] = pd.to_numeric(crypto_df['price_usd'])

crypto_df = crypto_df.sort_values(by = 'market_cap_usd', ascending = False)
crypto_df = crypto_df[crypto_df['symbol'].isin(exclusion_list) == False]#[:30]

crypto_df['weight'] = crypto_df['market_cap_usd']/crypto_df['market_cap_usd'].sum()
crypto_df['dollar_weight'] = crypto_df['weight']*capital
crypto_df['expected_holdings'] = crypto_df['dollar_weight']/crypto_df['price_usd']

crypto_list = [(lambda x: x.lower().replace(" ", "-"))(x) for x in list(crypto_df['name'])]

In [13]:
# Starting table: <table class="table">
# Ending table: </table>

# site: https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20130428&end=20171111

ticker = 'bitcoin'
start_date = '20130428'
end_date = '20171210'

def coinmktcapPull(ticker, start_date, end_date):
    
    wrongticker = "Sorry, we couldn't find your page"

    date_range = '/historical-data/?start=' + start_date + '&end=' + end_date
    site = 'https://coinmarketcap.com/currencies/' + ticker + date_range
    res = requests.get(site)
    soup = bs(requests.get(site).text, "lxml")
    #soup = soup.prettify()

    if wrongticker in soup:
        return ticker, ticker

    table_start = '<table class="table">'
    table_end = '</table>'
    if len(soup.find_all('table')) == 0:
        return ticker, ticker

    table = soup.find_all('table')[0]    

    # Parsing table from beautiful soup text
    row_marker = 0
    dates = []
    opens = []
    highs = []
    lows = []
    closes = []
    volumes = []
    mktcaps = []
    for row in table.find_all('tr'):
        column_marker = 0
        columns = str(row).split('\n')
        if 'Date' in columns[1]:
            continue
        else:
            dates = dates + [columns[1].split('<td class="text-left">')[1].split('</td>')[0]]
            opens = opens + [columns[2].split('<td>')[1].split('</td>')[0].replace(',','').replace('-','0')]
            highs = highs + [columns[3].split('<td>')[1].split('</td>')[0].replace(',','').replace('-','0')]
            lows = lows + [columns[4].split('<td>')[1].split('</td>')[0].replace(',','').replace('-','0')]
            closes = closes + [columns[5].split('<td>')[1].split('</td>')[0].replace(',','').replace('-','0')]
            volumes = volumes + [columns[6].split('<td>')[1].split('</td>')[0].replace(',','').replace('-','0')]
            mktcaps = mktcaps + [columns[7].split('<td>')[1].split('</td>')[0].replace(',','').replace('-','0')]
    '''
    crypto_dict = {ticker + '_open': opens,
                   ticker + '_high': highs,
                   ticker + '_low': lows,
                   ticker + '_close': closes,
                   ticker + '_volume': volumes,
                   ticker + '_mktcap': mktcaps}
    '''
    crypto_closes = {ticker: closes} #{ticker + '_close': closes}
    crypto_mktcaps = {ticker: mktcaps}#{ticker + '_mktcap': mktcaps}
    close_df = pd.DataFrame(crypto_closes, index = dates)
    mktcap_df = pd.DataFrame(crypto_mktcaps, index = dates)
    
    #coinmktcapdf[ticker + '_open'] = pd.to_numeric(coinmktcapdf[ticker + '_open'])
    #coinmktcapdf[ticker + '_high'] = pd.to_numeric(coinmktcapdf[ticker + '_high'])
    #coinmktcapdf[ticker + '_low'] = pd.to_numeric(coinmktcapdf[ticker + '_low'])
    close_df[ticker] = pd.to_numeric(close_df[ticker])
    #coinmktcapdf[ticker + '_volume'] = pd.to_numeric(coinmktcapdf[ticker + '_volume'])
    mktcap_df[ticker] = pd.to_numeric(mktcap_df[ticker])
    close_df.index = pd.to_datetime(close_df.index)
    mktcap_df.index = pd.to_datetime(mktcap_df.index)
    return close_df, mktcap_df

In [19]:
crypto_list = list(pd.read_csv('altcoinslist.csv')['Name'])

all_close = []
all_mktcap = []
wrong_names = []

start_date = '20130428'
end_date = '20171210'
for tick in crypto_list:
    temp_close, temp_mktcap = coinmktcapPull(tick, start_date, end_date)
    if type(temp_close) == str:
        wrong_names += [temp_close]
    else:
        all_close += [temp_close]
        all_mktcap += [temp_mktcap]
        
#correct_names = ['stellar', 'bytecoin-bcn','metaverse','byteball','attention-token-of-media','rlc']

#for tick in correct_names:
#    temp_close, temp_mktcap = coinmktcapPull(tick, start_date, end_date)
#    if type(temp_close) == str:
#        wrong_names += [temp_close]
#    else:
#        all_close += [temp_close]
#        all_mktcap += [temp_mktcap]

In [22]:
cc_mcaps.to_csv('marketcaps.csv')
cc_close.to_csv('closes.csv')

In [21]:
cc_close = pd.concat(filter(lambda x: type(x) != str, all_close), axis = 1).fillna(0)
cc_mcaps = pd.concat(filter(lambda x: type(x) != str, all_mktcap), axis = 1).fillna(0)

cc_index = cc_close[['bitcoin']]
cc_index.columns = ['index']
cc_returns = cc_close.pct_change().fillna(0)

In [21]:
eth_close,eth_mktcap = coinmktcapPull('ethereum', start_date, end_date)
eth_close.to_csv('eth.csv')

In [6]:
i = 0
top_picks = 10
last_month = int(cc_index.index[i].month)
index_start = 100

for idx, row in cc_index.iterrows():
    curr_month = int(idx.month)
    if i == 0:
        mcaps_temp = cc_mcaps.iloc[i].sort_values(ascending = False)[:top_picks]
        mcaps_temp = mcaps_temp[mcaps_temp != 0]
        wts = mcaps_temp/mcaps_temp.sum()
    elif curr_month != last_month:
        mcaps_temp = cc_mcaps.iloc[i].sort_values(ascending = False)[:top_picks]
        mcaps_temp = mcaps_temp[mcaps_temp != 0]
        wts = mcaps_temp/mcaps_temp.sum()
    else:
        pass
    
    temp_index = pd.concat([cc_returns.iloc[i],wts],axis = 1).dropna()
    temp_index.columns = ['returns','weights']
    idx_return = (temp_index['returns']*temp_index['weights']).sum()
    
    idx_val = idx_return
    
    cc_index.set_value(idx,'index', idx_val)
    
    last_month = curr_month
    i += 1
    
cc_index = cc_index.replace(np.inf, np.nan).fillna(0)    

In [7]:
def getccwts(exclusions, capital, top_n):
    
    response = requests.get('https://api.coinmarketcap.com/v1/ticker/')
    json_data = json.loads(response.text)
    i = 0
    crypto_df = {}
    for cc in json_data:
        temp_df = pd.DataFrame(cc, index = [i])[['symbol','name','market_cap_usd','price_usd']]
        if len(crypto_df) == 0:
            crypto_df = temp_df
        else:
            crypto_df = pd.concat([crypto_df,temp_df],axis = 0)
        i += 1

    crypto_df['market_cap_usd'] = pd.to_numeric(crypto_df['market_cap_usd'])
    crypto_df['price_usd'] = pd.to_numeric(crypto_df['price_usd'])

    crypto_df = crypto_df.sort_values(by = 'market_cap_usd', ascending = False)
    crypto_df = crypto_df[crypto_df['symbol'].isin(exclusion_list) == False][:top_n]

    crypto_df['weight'] = crypto_df['market_cap_usd']/crypto_df['market_cap_usd'].sum()
    crypto_df['dollar_weight'] = crypto_df['weight']*capital
    crypto_df['expected_holdings'] = crypto_df['dollar_weight']/crypto_df['price_usd']
    
    return crypto_df

In [12]:
    
exclusion_list = ['BCC','XRP','USDT','ETC']
capital = 15000
top_n = 30

test = getccwts(exclusion_list, capital, top_n)
test['price_btc'] = test['price_usd']/16219
test['btc_cost'] = test['price_btc']*test['expected_holdings']
test

,symbol,name,market_cap_usd,price_usd,weight,dollar_weight,expected_holdings,price_btc,btc_cost
0,BTC,Bitcoin,2.777880e+11,16600.800000,0.686193,10292.894875,0.620024,1.023540e+00,0.634620
1,ETH,Ethereum,4.429515e+10,460.137000,0.109418,1641.270761,3.566918,2.837024e-02,0.101194
2,BCH,Bitcoin Cash,2.332375e+10,1384.290000,0.057614,864.216050,0.624303,8.534990e-02,0.053284
3,MIOTA,IOTA,1.211203e+10,4.357580,0.029919,448.787544,102.990087,2.686713e-04,0.027670
5,LTC,Litecoin,8.332011e+09,153.593000,0.020582,308.726445,2.010029,9.469943e-03,0.019035
6,DASH,Dash,5.538181e+09,714.976000,0.013680,205.206527,0.287012,4.408262e-02,0.012652
7,BTG,Bitcoin Gold,4.491344e+09,268.948000,0.011095,166.418007,0.618774,1.658228e-02,0.010261
8,XMR,Monero,4.023443e+09,260.430000,0.009939,149.080870,0.572441,1.605709e-02,0.009192
9,XEM,NEM,3.713598e+09,0.412622,0.009173,137.600149,333.477491,2.544066e-05,0.008484
10,ADA,Cardano,2.886072e+09,0.111315,0.007129,106.937778,960.677159,6.863247e-06,0.006593
